## Cull Oracle TX
* See [0x90bc3e68ba8393a3bf2d79309365089975341a43](https://polygonscan.com/address/0x90bc3e68ba8393a3bf2d79309365089975341a43) for contract on polygon explorer

In [2]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from datetime import datetime
from pachira import *

In [3]:
POLY_POOL_CONTRACT = '0x90bc3e68ba8393a3bf2d79309365089975341a43'

In [4]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [12]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
sync_filt = Filter.create_filter(address=None, event_types=[Pair.events.Sync])  # Listen events from any smart contract

In [19]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [20]:
start = start - 100

In [21]:
rOracleEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=sync_filt)

In [22]:
events = {}
evt: LogResult
for k, evt in enumerate(rOracleEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])
    pair: PairDetails = FetchPairDetails().apply(web3, evt['address'])

    amt0 = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    amt1 = Conversion().convert_int256_bytes_to_int(arguments[1])  
    amt0_human = amt0/(10**pair.token0.decimals)
    amt1_human = amt1/(10**pair.token1.decimals)    

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt['event']
    event['details']['token0'] = pair.token0.address
    event['details']['token1'] = pair.token1.address
    event['details']['token0_symbol'] = pair.token0.symbol
    event['details']['token1_symbol'] = pair.token1.symbol 
    event['details']['token0_decimal'] = pair.token0.decimals
    event['details']['token1_decimal'] = pair.token1.decimals        
    event['details']['amount0'] = amt0
    event['details']['amount1'] = amt1
    event['details']['price'] = amt1_human/amt0_human
    
    events[k] = event
    if key not in processed_events and event['address'] == POLY_POOL_CONTRACT:
        print(f"Sync at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")

Sync at block:65,328,645 tx:0x5387c065a129f7e18ddec49cfccfadf52c4c1a1196e58c5a8d8283da3e05a993
.


In [24]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events = df_events[df_events['address'].values == POLY_POOL_CONTRACT]
df_events.sort_values(by=["blk_num"], ascending=[False], inplace=True)
df_events.reset_index(drop=True, inplace=True)

In [28]:
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,sync,sushi,0x90bc3e68ba8393a3bf2d79309365089975341a43,0x5387c065a129f7e18ddec49cfccfadf52c4c1a1196e5...,65328645,1733871028,{'web3_type': <class 'web3._utils.datatypes.Sy...


In [29]:
pd.DataFrame.to_dict(df_events, orient='index') 

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0x90bc3e68ba8393a3bf2d79309365089975341a43',
  'tx_hash': '0x5387c065a129f7e18ddec49cfccfadf52c4c1a1196e58c5a8d8283da3e05a993',
  'blk_num': 65328645,
  'timestamp': 1733871028,
  'details': {'web3_type': web3._utils.datatypes.Sync,
   'token0': '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619',
   'token1': '0xD6DF932A45C0f255f85145f286eA0b292B21C90B',
   'token0_symbol': 'WETH',
   'token1_symbol': 'AAVE',
   'token0_decimal': 18,
   'token1_decimal': 18,
   'amount0': 18395342747844443471,
   'amount1': 245833192469072045843,
   'price': 13.36388214336907}}}